In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as F
import torch.optim as optim

import os, sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))
from libs import data as dt, neuronshap as ns, sim
from cfgs.fedargs import *

from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    equalized_odds_ratio,
)
from libs.helpers.finance import bin_hours_per_week
from libs.helpers.metrics import (
    conditional_demographic_parity_difference,
    conditional_demographic_parity_ratio,
)
from libs.helpers.plot import group_box_plots

/Users/hkasyap/anaconda3/envs/menv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../../data/bank/bank.csv')

le = preprocessing.LabelEncoder()
cts_features = ['y', 'default', 'loan', 'housing', 'marital', 'balance', 'age', 'day', 'duration', 'campaign', 'pdays', 'previous']
for i in cts_features:
    df[i] = le.fit_transform(df[i])

one_hot_features = ['job', 'education', 'month', 'contact', 'poutcome']

df = pd.get_dummies(df, columns = one_hot_features)
    
#df['y'] = [1 if v == 'yes' else 0 for v in df['y']]
df = df.sample(frac = 1)
df.head()
#df.info()

,age,marital,default,balance,housing,loan,day,duration,campaign,pdays,...,month_nov,month_oct,month_sep,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
21533,15,0,0,867,1,0,16,407,0,0,...,1,0,0,1,0,0,0,0,0,1
37947,16,0,0,3732,1,1,7,215,2,218,...,0,0,0,1,0,0,1,0,0,0
11890,28,0,0,1242,0,0,8,130,0,0,...,0,0,0,1,0,0,0,0,0,1
7273,15,0,0,1338,1,0,1,214,1,0,...,0,0,0,0,0,1,0,0,0,1
19811,17,0,0,2123,1,0,21,73,6,0,...,0,0,0,1,0,0,0,0,0,1


In [3]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
train_oh, test_oh = train_df, test_df
cts_features = ['balance', 'age', 'day', 'duration', 'campaign', 'pdays', 'previous']

ss = StandardScaler()

train_oh[cts_features] = ss.fit_transform(train_df[cts_features])
test_oh[cts_features] = ss.fit_transform(test_df[cts_features])

#test_df['agp'] = [1 if v < 25 or v > 60 else 0 for v in test_df['age']]

In [4]:
test_oh.head()

,age,marital,default,balance,housing,loan,day,duration,campaign,pdays,...,month_nov,month_oct,month_sep,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
36387,1.854574,0,0,-0.655206,0,0,0.619094,-0.164457,0.069181,0.502786,...,0,0,0,1,0,0,0,0,1,0
23176,1.095455,0,0,2.647001,0,0,0.499071,-0.135166,-0.573722,-0.413956,...,1,0,0,1,0,0,0,0,0,1
7364,-0.422782,1,0,-0.730063,1,0,-1.541314,-0.340204,-0.573722,-0.413956,...,0,0,0,0,0,1,0,0,0,1
33545,-0.992122,1,0,-0.499841,1,0,-0.221065,-0.243962,0.390632,-0.413956,...,0,0,0,1,0,0,0,0,0,1
14130,-1.181901,1,0,-0.046458,0,1,0.739116,0.898396,-0.252270,-0.413956,...,0,0,0,1,0,0,0,0,0,1


In [5]:
#https://github.com/tailequy/fairness_dataset/blob/main/experiments/Fair-metrics.ipynb

In [6]:
m_dh_oh = test_oh.loc[test_oh["marital"] == 1]
m_dh_oh = m_dh_oh.head(100)
fm_dh_oh = test_oh.loc[test_oh["marital"] == 0]
fm_dh_oh = fm_dh_oh.head(100)

In [7]:
X_train = train_oh.drop(columns="y").values
Y_train = train_oh['y'].values
X_test = test_oh.drop(columns="y").values
Y_test = test_oh['y'].values
X_m = m_dh_oh.drop(columns="y").values
Y_m = m_dh_oh['y'].values
X_fm = fm_dh_oh.drop(columns="y").values
Y_fm = fm_dh_oh['y'].values

#creating torch dataset and loader using original dataset. 
#to use resampled dataset, replace ex. xtrain with xtrain_over etc.
train_data = torch.utils.data.TensorDataset(torch.tensor(X_train).float(), torch.tensor(Y_train).long())
test_data = torch.utils.data.TensorDataset(torch.tensor(X_test).float(), torch.tensor(Y_test).long())
m_data = torch.utils.data.TensorDataset(torch.tensor(X_m).float(), torch.tensor(Y_m).long())
fm_data = torch.utils.data.TensorDataset(torch.tensor(X_fm).float(), torch.tensor(Y_fm).long())

train_loader = torch.utils.data.DataLoader(train_data,batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=len(test_data))
m_loader = torch.utils.data.DataLoader(m_data, batch_size=1)
fm_loader = torch.utils.data.DataLoader(fm_data, batch_size=1)

In [8]:
class BasicNet(torch.nn.Module):
    
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_features = num_features
        self.num_classes = num_classes
        self.layers = 0
        
        self.lin1 = torch.nn.Linear(self.num_features,  150)        
        self.lin2 = torch.nn.Linear(50, 50)        
        self.lin3 = torch.nn.Linear(50, 50)
        
        self.lin4 = torch.nn.Linear(150, 150) 
        
        self.lin5 = torch.nn.Linear(50, 50)        
        self.lin6 = torch.nn.Linear(50, 50)
        self.lin10 = torch.nn.Linear(150, self.num_classes)
        
        self.prelu = torch.nn.PReLU()
        self.dropout = torch.nn.Dropout(0.25)

    def forward(self, xin):
        self.layers = 0
        
        x = F.relu(self.lin1(xin))
        self.layers += 1
        
        #x = F.relu(self.lin2(x))
        #self.layers += 1
        for y in range(8):
            x = F.relu(self.lin4(x)) 
            self.layers += 1
           
        x = self.dropout(x)
        
        x = F.relu(self.lin10(x)) 
        self.layers += 1
        return x

In [9]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    
    for inputs, target in train_loader:
      
        #inputs, target = inputs.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target.long())
        # Backprop
        loss.backward()
        optimizer.step()
        ###

In [10]:
def test(model, test_loader):
    model.eval()
    
    test_loss = 0
    correct = 0
    test_size = 0
    
    with torch.no_grad():
      
        for inputs, target in test_loader:
            
            #inputs, target = inputs.to(device), target.to(device)
            
            output = model(inputs)
            test_size += len(inputs)
            test_loss += test_loss_fn(output, target.long()).item() 
            pred = output.max(1, keepdim=True)[1] 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= test_size
    accuracy = correct / test_size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, test_size,
        100. * accuracy))
    
    return test_loss, accuracy

In [11]:
model = BasicNet(46, 2)
test_accuracy = []
train_loss = []
nbr_epochs = 5
lr = 0.0025# 
weight_decay = 0

# Surrogate loss used for training
loss_fn = torch.nn.CrossEntropyLoss()
test_loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')

optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=lr ,weight_decay=weight_decay)
#optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)

print('Training beginning...')
#start_time = time.time()

for epoch in range(1, nbr_epochs+1):
    print('Epoch ', epoch, ':')
    train(model, train_loader, optimizer, epoch)
    loss, acc = test(model, test_loader)
    
    # save results every epoch
    test_accuracy.append(acc)
    train_loss.append(loss)
    
#end_time = time.time()
#print('Training on ' + str(nbr_epochs) + ' epochs done in ', str(end_time-start_time),' seconds')

Training beginning...
Epoch  1 :

Test set: Average loss: 0.6931, Accuracy: 10590/12002 (88%)

Epoch  2 :

Test set: Average loss: 0.2832, Accuracy: 10590/12002 (88%)

Epoch  3 :

Test set: Average loss: 0.2185, Accuracy: 10759/12002 (90%)

Epoch  4 :

Test set: Average loss: 0.2178, Accuracy: 10790/12002 (90%)

Epoch  5 :

Test set: Average loss: 0.2164, Accuracy: 10840/12002 (90%)



In [12]:
with torch.no_grad():
    for inputs, target in test_loader:
        outputs = model(inputs)
        pred = outputs.max(1, keepdim=True)[1] 
        correct = pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / len(inputs)
        print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, len(inputs), 100. * accuracy))

Y_prob = F.softmax(outputs, dim=1)[:, 1]
Y_pred = outputs.max(1, keepdim=True)[1]

print(sum(Y_test), sum(Y_pred), sum(pred))
test_df['agp'] = [1 if v < 25 or v > 60 else 0 for v in test_df['age']]


Accuracy: 10840/12002 (90%)

1412 tensor([1346]) tensor([1346])


<h1>Demographic Parity</h1>

<h2>Distribution of scores by sex</h2>

In [13]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test_oh.marital,
)

dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test_oh.marital,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.062
Demographic parity ratio: 0.601


<h2>Distribution of scores by age</h2>

In [14]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test_df.agp,
)
dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test_df.agp,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.000
Demographic parity ratio: 1.000


<h1>Conditional Demographic Parity</h1>

<h2>Distribution of scores by sex and hours worked per week</h2>

In [15]:
test_hpw_enum = test.hours_per_week.map(bin_hours_per_week)

cdpd = conditional_demographic_parity_difference(
    Y_test, Y_pred, test.sex, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    Y_test, Y_pred, test.sex, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

AttributeError: 'function' object has no attribute 'hours_per_week'

<h2>Distribution of scores by race and hours worked per week</h2>

In [ ]:
cdpd = conditional_demographic_parity_difference(
    Y_test, Y_pred, test.race, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    Y_test, Y_pred, test.race, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

<h1>Equalised Odds</h1>

<h2>Distribution of scores by sex for high and low earners</h2>

In [ ]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test_oh.marital,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test_oh.marital,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

<h2>Distribution of scores by race for high and low earners</h2>

In [ ]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test_oh.MARRIED,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test_oh.MARRIED,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

<h1>Shapley based Neuron Pruning for Fairness</h1>

In [16]:
m_shapley_values = ns.calculate_shapley_values_fa(model, m_loader, 100)
print(m_shapley_values)
fm_shapley_values = ns.calculate_shapley_values_fa(model, fm_loader, 100)
print(fm_shapley_values)

[  1.6675439   5.801022    0.        ... 523.1783     88.86537
   0.       ]
[1.5468395e+00 0.0000000e+00 3.9565407e-02 ... 4.8433694e+02 5.5896687e+01
 0.0000000e+00]


In [17]:
diff_shap_values = np.abs(m_shapley_values - fm_shapley_values)
max_diff_shap_values_ind = np.argpartition(diff_shap_values, -90)[-90:]
diff_shap_values[max_diff_shap_values_ind]

array([12.556049 , 12.602154 , 12.879402 , 12.940365 , 13.161552 ,
       13.83736  , 14.460541 , 13.903233 , 13.34423  , 14.329932 ,
       13.263633 , 13.401623 , 13.771339 , 14.467682 , 14.569092 ,
       14.605108 , 14.795906 , 15.917956 , 15.204925 , 16.014202 ,
       16.072803 , 15.562981 , 14.953419 , 15.46846  , 15.110779 ,
       15.694984 , 15.509811 , 15.330742 , 14.809639 , 14.9985075,
       15.933289 , 14.976845 , 15.928108 , 16.117401 , 16.715935 ,
       18.114815 , 17.24144  , 17.11908  , 16.542168 , 17.369225 ,
       18.026913 , 17.747461 , 16.274342 , 17.57302  , 16.745995 ,
       16.63903  , 17.985737 , 16.671036 , 16.906916 , 18.200851 ,
       23.880348 , 18.670265 , 19.284828 , 22.259628 , 24.411934 ,
       79.13246  , 18.796112 , 23.284752 , 20.495018 , 29.796738 ,
       29.405807 , 27.879044 , 21.046402 , 22.737404 , 25.517414 ,
       20.84382  , 20.325775 , 29.212692 , 43.054962 , 36.008373 ,
       20.407143 , 37.231995 , 22.131805 , 25.11827  , 18.6479

In [18]:
model_arr, model_slist = sim.get_net_arr(model)
model_arr[max_diff_shap_values_ind] = 0
updated_model = sim.get_arr_net(model, model_arr, model_slist)

In [19]:
with torch.no_grad():
    for inputs, target in test_loader:
        outputs = updated_model(inputs)
        pred = outputs.max(1, keepdim=True)[1] 
        correct = pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / len(inputs)
        print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, len(inputs), 100. * accuracy))
        

Y_prob = F.softmax(outputs, dim=1)[:, 1]
Y_pred = outputs.max(1, keepdim=True)[1]
print(sum(Y_test), sum(Y_pred), sum(pred))


Accuracy: 10819/12002 (90%)

1412 tensor([831]) tensor([831])


In [20]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test_oh.marital,
)
dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test_oh.marital,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.029
Demographic parity ratio: 0.677


In [ ]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test_df.agp,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test_df.agp,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")